# WoW Stats (API Returns Inconsistent Values Between Methods)

Pipeline
1. Get leaderboards
2. Get guilds in leaderboards
3. Get characters in guilds in leaderboards
4. Get player stats from leaderboards

Rinse repeat with achievements

## Initialize Default Values and Functions

#### Import Packages

In [ ]:
import requests
import configparser
import json
import os
import pandas as pd
import csv
import glob
from IPython.display import HTML
from datetime import datetime
import numpy as np


#### Define reusable functions and get API keys

In [ ]:
def unpack_json(txt):
    """Converts json data from API calls into dicts and lists for parsing."""
    unpacked = json.loads(txt)
    return unpacked

def get_access_token (blizzard_key, blizzard_secret):
    """Gets a Blizzard access token for use with the Blizzard API."""
    r = requests.post('https://us.battle.net/oauth/token', data={'grant_type': 'client_credentials'},
                  auth=(blizzard_key, blizzard_secret))
    unpacked = unpack_json(r.text)
    access_token = unpacked['access_token']
    return access_token

# Read in keys and secret IDs for APIs from a separate conig file
f_config = 'config.ini'
config = configparser.ConfigParser()
config.read(f_config)
blizzard_key = config.get('KEYS', 'blizzard')
blizzard_secret = config.get('KEYS', 'blizzard_secret')
access_token = get_access_token(blizzard_key, blizzard_secret)
locale = 'en_US'
namespace = 'static-us'

#### Get a list of all the US-based World of Warcraft Servers and IDs 

In [ ]:
def get_wow_realms_list (namespace, locale):
    directory = 'data/wow/realm/index'
    url = 'https://us.api.blizzard.com/' + directory + '?namespace=dynamic-us' + \
          '&locale=' + locale + '&access_token=' + access_token
    r = requests.get(url)
    unpacked = unpack_json(r.text)
    realm_names = []
    realm_ids = []
    realm_slugs = []
    for realm in unpacked ['realms']:
        realm_names.append(realm['name'])
        realm_ids.append(realm['id'])
        realm_slugs.append(realm['slug'])
    return realm_names, realm_ids, realm_slugs

realm_names,realm_ids,slugs = get_wow_realms_list (namespace, locale)
for i,slug in enumerate(slugs):
    print(realm_ids[i], slug)

## Mythic Dungeons
This works really well for old seasons (period id <200) but BFA dungeon data does not include player roster. Only an id, a map, and dungeon name, so it's not useful anymore.

In [ ]:

def get_mythic_dungeon_index (access_token):
    url = 'https://us.api.blizzard.com/data/wow/mythic-keystone/dungeon/index?namespace=dynamic-us&locale=en_US&access_token=' + access_token
    r = requests.get(url)
    unpacked = unpack_json(r.text)
    dungeon_ids = []
    dungeon_names = []
    for dungeon in unpacked['dungeons']:
        dungeon_ids.append(dungeon['id'])
        dungeon_names.append(dungeon['name'])
    return dungeon_ids, dungeon_names
    
    
def get_keystone_leaderboard (realm_id, period, dungeon_id, access_token):
    df = pd.DataFrame()
    url = 'https://us.api.blizzard.com/data/wow/connected-realm/' + str(realm_id) + '/mythic-leaderboard/'+ str(dungeon_id) + '/period/' + str(period) + '?namespace=dynamic-us&locale=en_US&access_token=' + access_token
    r = requests.get(url)
    try:
        unpacked = unpack_json(r.text)
        for leading_group in unpacked['leading_groups']:
            for member in leading_group['members']:
                row = dict(member_name = member['profile']['name'], member_id = member['profile']['id'], 
                      member_realm = member['profile']['realm']['slug'], member_realm_id = member['profile']['realm']['id'],
                      member_faction = member['faction']['type'], member_specialization = member['specialization']['id'],
                      realm_id = realm_id, period = period, 
                      ranking = leading_group['ranking'],duration = leading_group['duration'],
                      completed_timestamp = leading_group['completed_timestamp'])
                df = df.append(row, ignore_index = True)
    except:
        pass
    df.to_csv('miracle.csv')
    return df
    
def get_periods_per_season(season_id):
    url = 'https://us.api.blizzard.com/data/wow/mythic-keystone/season/' \
          + str(season_id) + '?namespace=dynamic-us&locale=en_US&access_token=' + access_token
    r = requests.get(url)
    unpacked = unpack_json(r.text)
    season_start = unpacked['start_timestamp']
    season_end = unpacked['end_timestamp']
    season_periods = []
    for period in unpacked['periods']:
        season_periods.append(period['id'])
    return season_periods


dungeon_ids, dungeon_names = get_mythic_dungeon_index(access_token)
periods_per_season = get_periods_per_season(1)

#for period in periods_per_season:
#    for dungeon in dungeon_ids:
#        for realm in realm_ids:
#            period_leaderboard = get_keystone_leaderboard (realm, period, dungeon, access_token)


## Build Leaderboard Datasets from Dataforazeroth.com

In [ ]:
def dataforazeroth (dir_dataset):
    date_cols = []
    os.chdir(dir_dataset)
    files = os.listdir()
    csv_files = glob.glob('*.{}'.format('csv'))
    df = pd.DataFrame()
    for f in csv_files:
        temp = pd.read_csv(f)
        temp.columns = ['ranking','leaderboard','player','guild','realm','score']
        temp.leaderboard = f.split('wow_')[1].split('.csv')[0]
        df = df.append(temp[['ranking','leaderboard','player','guild','realm']], ignore_index=True)
    df.to_csv('../dataforazeroth_complete_dataset.csv')
    return df
    
dir_dataset = '/Users/haleyspeed/Docs/insight/datasets/dataforazeroth_datasets'
dataforazeroth_set = dataforazeroth (dir_dataset)
display(HTML(dataforazeroth_set.head().to_html()))

## Get Guild Rosters from Leaderboards

In [ ]:
def get_guild_roster (realm, guild, access_token):
    df = pd.DataFrame()
    url = 'https://us.api.blizzard.com/data/wow/guild/'+ realm \
          + '/' + guild + '/roster?namespace=profile-us&locale=en_US&access_token=' + access_token
    r = requests.get(url)
    unpacked = unpack_json(r.text)
    if r.status_code != 200:
        return df
    else:
        try:
            guild_faction = unpacked['guild']['faction']['name']
            for member in unpacked['members']:
                row = dict(player = member['character']['name'], id = member['character']['id'],
                      realm = member['character']['realm']['slug'], realm_id = member['character']['realm']['id'],
                      level = member['character']['level'], playable_class = member['character']['playable_class']['id'],
                      playable_race = member['character']['playable_race']['id'], guild_rank = member['rank'],
                      guild_name = guild, faction = guild_faction)
                df = df.append(row, ignore_index = True)
        except:
            return df
    return df

roster = pd.DataFrame()
realms = []
guilds = []
dfa_set = pd.read_csv('/Users/haleyspeed/Docs/insight/datasets/dataforazeroth_complete_dataset.csv')
dfa_set['player_id'] = ''
i = 4601
print(os.getcwd())
for row in dfa_set.iloc[4600:][:].itertuples():
    try:
        realm_slug = row.realm.replace('US-', '').replace("'",'').replace(' ', '-').lower()
        player_slug = row.player.lower()
        dfa_set.at[i,'player_id'] = player_slug + '-' + realm_slug
        print(i, end = ' ')
        if isinstance(row.guild, float): 
            continue
        if row.realm in realms and row.guild in guilds:
            continue
        guild_slug = row.guild.replace(' ','-').lower()
        realms.append(realm_slug)
        guilds.append(guild_slug)
        roster = roster.append(get_guild_roster (realm_slug, guild_slug, access_token), ignore_index = True)
        if i % 100 == 0: # Every 100 guilds, dump the data into a file 
            roster.to_csv('wow_roster' + str(i) + '.csv')
            roster = pd.DataFrame()  # clear memory by resetting the dataframe to empty
        i = i + 1
    except:
        continue


## Get Profiles from Leaderboards

In [ ]:
def get_wow_profile (realm, player, token):
    url = 'https://us.api.blizzard.com/profile/wow/character/' + realm \
          + '/' + player + '?namespace=profile-us&locale=en_US&access_token=' + access_token
    r = requests.get(url)
    unpacked = unpack_json(r.text)
    row = dict(id = unpacked['id'], name = unpacked['name'], gender = unpacked['gender']['name'],
          faction = unpacked['faction']['name'], race = unpacked['race']['name'],
          character_class = unpacked['character_class']['name'],
          active_spec = unpacked['active_spec']['name'], realm = unpacked['realm']['slug'],
          guild = unpacked['guild']['name'], level = unpacked['level'],
          achievement_points = unpacked['achievement_points'],
          last_login = unpacked['last_login_timestamp'],
          average_item_level = unpacked['average_item_level'],
          equipped_item_level = unpacked['equipped_item_level'])
    return row



In [ ]:
print (600 %200)